In [ ]:
%%capture
import pandas as pd
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
# Adiciona o primeiro dataframe

culturas=pd.read_csv("/content/drive/MyDrive/AED_trabalho_final/tabela5457_producao_ton_mun_10.csv",sep=';',skiprows=4)
culturas=culturas.iloc[:len(culturas)-22] #remove as legendas do rodapé
culturas

,Cód.,Município,Abacate,Abacaxi*,Açaí,Algodão arbóreo (em caroço),Algodão herbáceo (em caroço),Alho,Amendoim (em casca),Arroz (em casca),Aveia (em grão),Azeitona
0,1100015,Alta Floresta D'Oeste (RO),-,100,20,-,-,-,8,405,-,-
1,1100023,Ariquemes (RO),-,416,50,-,-,-,-,3120,-,-
2,1100031,Cabixi (RO),-,55,-,-,-,-,1,4320,-,-
3,1100049,Cacoal (RO),-,1955,-,-,-,-,5,24,-,-
4,1100056,Cerejeiras (RO),-,505,-,-,-,-,6,748,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...
5558,5222005,Vianópolis (GO),-,-,-,-,-,-,-,-,-,-
5559,5222054,Vicentinópolis (GO),-,-,-,-,-,-,-,-,-,-
5560,5222203,Vila Boa (GO),-,-,-,-,-,-,-,-,-,-
5561,5222302,Vila Propício (GO),-,-,-,-,-,-,-,10,-,-


In [ ]:
# Baixei todas as culturas de 2022. Será feita o merge para criar um dataframe único chamado cultura

compilados=[20,30,40,50,60]

for compilado in compilados:
    df = pd.read_csv(f"/content/drive/MyDrive/AED_trabalho_final/tabela5457_producao_ton_mun_{compilado}.csv", sep=';', skiprows=4)
    df = df.iloc[:len(df) - 22]  # remove as legendas do rodapé
    df=df.drop(columns=["Município"]) #derruba município para evitar ficar acumulando pra sempre entre um merge e outro

    culturas = pd.merge(culturas, df, on="Cód.", how="left")

culturas.shape

(5563, 67)

In [ ]:
# Cria coluna para mostrar apenas a maior cultura de cada ano

# Atenção: quando a cultura não está disponível naquele município, o valor retornado é um "-" ou "...".
# Por isso, é necessário substituir esses valores por ZERO.
culturas.iloc[:, 2:67] = culturas.iloc[:, 2:67].replace({'-': 0, "...":0}).astype(int)

# Cria coluna 'Principal' com a cultura que tem o valor máximo em cada linha
culturas["Principal"] = culturas.iloc[:, 2:67].idxmax(axis=1)
culturas["Peso"] = culturas.iloc[:, 2:67].max(axis=1)

# Exporta para .csv
#culturas.to_csv("/content/drive/MyDrive/AED_trabalho_final/culturas_mun_novo.csv")

<ipython-input-9-9e39c7c43cc1>:5: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  culturas.iloc[:, 2:67] = culturas.iloc[:, 2:67].replace({'-': 0, "...":0}).astype(int)


In [ ]:
culturas.to_csv("/content/drive/MyDrive/AED_trabalho_final/culturas_mun_novo.csv")

# Relaciona o nível de seca em dezembro de 2022 a cultura principal do município

In [ ]:
# Dados de seca (dez/2022)
seca=pd.read_excel("/content/drive/MyDrive/AED_trabalho_final/IIS3_IIS6_Brasil_2022_12.xlsx",usecols=["Geo_cod","UF","Regiao","IIS6_1222"])

# Renomeia a coluna Geo_Cod para Cód.
seca=seca.rename(columns={"Geo_cod": "Cód."})
seca

,Cód.,UF,Regiao,IIS6_1222
0,1100015,RO,Norte,5.0
1,1100023,RO,Norte,3.0
2,1100031,RO,Norte,5.0
3,1100049,RO,Norte,6.0
4,1100056,RO,Norte,5.0
...,...,...,...,...
5567,5222005,GO,Centro-Oeste,5.0
5568,5222054,GO,Centro-Oeste,4.0
5569,5222203,GO,Centro-Oeste,6.0
5570,5222302,GO,Centro-Oeste,5.0


In [ ]:
# Relaciona nível de seca com as principais culturas
principal_cultura = culturas[["Cód.","Município","Principal"]] # seleciona colunas de interesse
principal_cultura["Cód."]=principal_cultura["Cód."].astype(int) # converte para int

compara_seca=pd.merge(principal_cultura,seca,how="left",on="Cód.")

# Deixa apenas os municípios em situação de seca hídrica
# O IIS possui as seguintes classes: condição normal (6), seca fraca (5), seca moderada (4), seca severa (3), seca extrema (2) e seca excepcional (1).
# Por isso, os únicos municípios que ficarão serão os em condição de seca, independente do volume de seca

compara_seca=compara_seca[compara_seca["IIS6_1222"] <=5]
compara_seca.Principal.value_counts()

<ipython-input-89-3a50b37760b3>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  principal_cultura["Cód."]=principal_cultura["Cód."].astype(int) # converte para int


Cana-de-açúcar                  1024
Soja (em grão)                   586
Milho (em grão)                  577
Mandioca                         371
Banana (cacho)                   148
Café (em grão) Total             127
Arroz (em casca)                 108
Trigo (em grão)                   94
Laranja                           56
Tomate                            50
Abacate                           46
Batata-inglesa                    28
Uva                               27
Tangerina                         24
Melancia                          20
Feijão (em grão)                  19
Cebola                            18
Coco-da-baía*                     15
Manga                             13
Abacaxi*                          12
Açaí                              12
Melão                             11
Batata-doce                       10
Maçã                               9
Erva-mate (folha verde)            8
Cacau (em amêndoa)                 8
Caqui                              7
F